# The Monge-Ampere equation

This notebook presents a series of methods for numerically solving the Monge-Ampere equation
$$
    \det(\nabla^2 u) = f
$$
with Dirichlet boundary conditions. 
For that purpose, we rely on the monotone and consistent MA-LBR numerical scheme (Monge-Ampere using Lattice Basis Reduction), and some variants.

*Summary of this series of notebooks:*
[Adaptive grid discretizations, summary](http://nbviewer.jupyter.org/urls/rawgithub.com/Mirebeau/AdaptiveGridDiscretizations/master/Notebooks/Summary.ipynb)

**References.**

The MA-LBR scheme was first introduced in:
* Benamou, J.-D., Collino, F., & Mirebeau, J.-M. (2016). Monotone and Consistent discretization of the Monge-Ampere operator. Mathematics of Computation, 85(302), 2743–2775.

**Reformulation as an extremal operator.**
The starting point of these methods is to observe that, for a positive definite $d \times d$ matrix $M$
$$
    d ({\rm det} M)^\frac 1 d = \inf_{{\rm det} D = 1} {\rm tr}(D M),
$$
where, implicitly, the optimization variable $D$ is also assumed to be a symmetric positive definite matrix.

If $M = \nabla^2 u$ is a hessian matrix, then the l.h.s. is (a multiple of a power of) the Monge-Ampere operator, while the r.h.s. is an infimum of second order linear operators, which can be discretized using monotone finite differences.

**Discretization strategy**
The chosen finite difference scheme for the linear operator is described in 
[I Tensor decomposition, dimensions 2 and 3](http://nbviewer.jupyter.org/urls/rawgithub.com/Mirebeau/AdaptiveGridDiscretizations/master/Notebooks/TensorSelling.ipynb)
Since 


## 0. Importing the required libraries

In [50]:
import sys; sys.path.append("..") # Allow import from parent directory
from NumericalSchemes import Selling
from NumericalSchemes import LinearParallel as lp
from NumericalSchemes import FiniteDifferences as fd
spad = fd.spAD # Alternatively : from NumericalSchemes import SparseAutomaticDifferentiation as spad

In [235]:
import numpy as np
import scipy.linalg; import scipy.sparse; import scipy.sparse.linalg 
from matplotlib import pyplot as plt

Some utility functions

In [ ]:
import importlib
fd = importlib.reload(fd)
spad = importlib.reload(spad)
spAD = spad.spAD

In [ ]:
1. Naive discretization

In [209]:
# Collect some superbases
e0,e1 = np.identity(2).astype(int)
SB0 = [ [e0,e,-e0-e] for e in (e1,-e1)] #[ [e0,e1,-e0-e1], [e0,-e1,e1-e0] ]
SB1 = SB0 + [ [e,e+f,-2*e-f] for e in (e0,-e0) for f in (e1,-e1) ]
SB0, SB1 = (np.array(S).transpose(2,1,0) for S in (SB0,SB1))

In [4]:
#Define the domain [0,1]^2, sampled on a cartesian grid
gridScale = 0.2
aX = np.arange(-1,1,gridScale); aY = aX
X,Y = np.meshgrid(aX,aY,indexing='ij')
bounds=X.shape

## 1. The AD-LBR scheme

$$
    \Lambda u(x) := \inf_{(e_0,e_1,e_2) \in S} H( \Delta_{e_0}^h u(x), \Delta_{e_1}^h u(x), \delta_{e_2}^h u(x))
$$
where $H(a,b,c) := H_0(a_+,b_+,c_+)$, and 
$$
    H_0(a,b,c) :=
    \begin{cases}
        b c \quad \text{ if }\ a \geq b+c, \text{ and likewise permuting } a,b,c\\
        \frac 1 2 (a b+b c+c a) - \frac 1 4 (a^2+b^2+c^2)\quad \text{ otherwise}
    \end{cases}
$$

In [317]:
np.maximum(np.ones((2,2)),2)

array([[2., 2.],
       [2., 2.]])

In [15]:
def H0(diffs): 
    # Interior case
    a,b,c = diffs
    result = 0.5*(a*b+b*c+c*a)-0.25*(a**2+b**2+c**2) 
    
    # Boundary case
    c,b,a=np.sort(diffs,axis=0) # This a>=max(b,c)
    pos = a>=b+c 
    result[pos] = (b*c)[pos]
    return result

def H(diffs):
    return H0(np.clip(diffs,0,None))

def Diff2(u,offset,gridScale):    
    return (np.roll(u,tuple(offset),axis=(0,1))+np.roll(u,tuple(-offset),axis=(0,1))-2*u)/gridScale**2

def MALBR(u,SB,gridScale):
    values = np.array([H(np.array([Diff2(u,offset,gridScale) for offset in sb])) 
                       for sb in SB])
    arg = np.argmin(values,axis=0)
    return (np.take_along_axis(values,np.expand_dims(arg,axis=0),axis=0), 
            SB[arg])

In [16]:
def Diff2_NonUniform(u,offsets,gridScale):
    bounds=u.shape
    grid = np.mgrid[tuple(slice(0,n) for n in bounds)]
    
    return u #TODO

def JacMALBR(u,sb,gridScale):
    # Returns a jacobian matrix, made of triplets
    diffs = np.array((Diff2_NonUniform(u,offset) for offset in sb))
    return 

def GradH(diffs): 
    """Gradient of the elementary function in the MA-LBR scheme.
    Assumes the entries are positive. (Otherwise no differentiability.)"""
    # Interior case
    a,b,c = diffs
    grad = np.array((b+c-a,c+a-b,a+b-c))
    
    # Boundary case
    indices = np.argsort(diffs,axis=0)
    c,b,a = np.take_along_axis(diffs,indices,axis=0)
    pos = a>=b+c
    np.put_along_axis( grad[:,pos], (b[pos],c[pos],np.full(a[pos].shape,0.)), 
                      indices[pos], axis=0 )

In [23]:
u=0.5*(X**2+4*Y**2+2*1.5*X*Y)
#MALBR(u,SB1,gridScale);
val,sb = MALBR(u,SB0,gridScale);

grid =np.mgrid[0:10,0:10];
np.broadcast_to(np.reshape(grid,

(2, 10, 10)

In [37]:
(np.full((1,2,1),0)+np.full((3,1,3),1)).shape

(3, 2, 3)

In [35]:
u[grid[0],grid[1]]-u

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [29]:
u[((0,1,2),(2,3,4)),((0,1,2),(6,7,8))]

array([[4.  , 2.56, 1.44],
       [0.08, 0.16, 0.56]])

In [28]:
u[1,1]

2.5600000000000005

In [17]:
u = X**2
Diff2(u,np.array((1,0)),gridScale)

array([[-18., -18., -18., -18., -18., -18., -18., -18., -18., -18.],
       [  2.,   2.,   2.,   2.,   2.,   2.,   2.,   2.,   2.,   2.],
       [  2.,   2.,   2.,   2.,   2.,   2.,   2.,   2.,   2.,   2.],
       [  2.,   2.,   2.,   2.,   2.,   2.,   2.,   2.,   2.,   2.],
       [  2.,   2.,   2.,   2.,   2.,   2.,   2.,   2.,   2.,   2.],
       [  2.,   2.,   2.,   2.,   2.,   2.,   2.,   2.,   2.,   2.],
       [  2.,   2.,   2.,   2.,   2.,   2.,   2.,   2.,   2.,   2.],
       [  2.,   2.,   2.,   2.,   2.,   2.,   2.,   2.,   2.,   2.],
       [  2.,   2.,   2.,   2.,   2.,   2.,   2.,   2.,   2.,   2.],
       [  2.,   2.,   2.,   2.,   2.,   2.,   2.,   2.,   2.,   2.]])

In [39]:
tuple(np.array([i**2 for i in range(3)]))

(0, 1, 4)